# Twitch Bot

In [221]:
import logging
import socket
import pandas as pd
import numpy as np
from emoji import demojize
import Levenshtein as lev
import re
from datetime import datetime

In [204]:
def levCalclulate(str1, str2):
    Distance = lev.distance(str1, str2)
    Ratio = lev.ratio(str1, str2)
#     print("Levenshtein entre {0} et {1}".format(str1, str2))
#     print("> Distance: {0}\n> Ratio: {1}\n".format(Distance, Ratio))
    return Distance, Ratio

#### BDD Anime

In [205]:
bdd = pd.read_excel(r'BDD\bdd_blindtest_v1.xlsx')

def remove_all_extra_spaces(string):
    return " ".join(string.split())

bdd = bdd.apply(lambda x: x.astype(str).str.lower())
bdd.loc[bdd["Type du son (EP,ED,OST)"] == "ost", "Numéro du son"] = 0
bdd["Numéro du son"] = bdd["Numéro du son"].astype(float).astype(int)

#### Barème
- Nom de l'oeuvre : +3 pts
- Titre du son : +1 pts
- Type du son : + 0.5 pts
- Numéro du son : + 0.5 pts
- Auteur du son : +1 pts

#### DataFrame avec une série de 10 questions aléatoire prisent depuis la base de donnée.

In [206]:
# Nombre de questions par séries
nb_q = 10

In [207]:
import random

In [208]:
liste_random = []

for i in range(nb_q): liste_random.append(random.randint(0, len(bdd)))

df_q = bdd.iloc[liste_random]
df_q.index = np.arange(1, len(df_q)+1)
df_q

,Catégorie,Nom de l'oeuvre,Titre du son,"Type du son (EP,ED,OST)",Numéro du son,Auteur du son
1,anime,shingeki no kyojin s2,shinzou wo sasageyo!,op,1,linked horizon
2,anime,naruto,naruto main theme,ost,0,toshio masuda
3,anime,one piece,we are! (remix),op,10,tvxq!
4,anime,naruto,seishun kyousoukyoku,op,5,sambomaster
5,anime,code geass - lelouch of the rebellion o.s.t. 2,innocent days,ost,0,"kōtarō nakagawa, hitomi kuroishi"
6,anime,fullmetal alchemist brotherhood,hologram,op,2,nico touches the walls
7,anime,bleach,dtecnolife,op,2,uverworld
8,anime,code geass : lelouch of the rebellion r2,o2,op,1,orange range
9,anime,code geass: lelouch of the rebellion,all hail britannia!!!,ost,0,"kōtarō nakagawa, hitomi kuroishi"
10,anime,naruto shippuden,utakata hanabi,ed,14,supercell


In [332]:
df_q_save = df_q

#### Tableau participant
A voir plus tard comment ajouter les participants sur cette liste.
Faire en sorte que la liste se met à jour automatiquement avec les participants sur le tchat twitch.

In [457]:
df_q["Numéro du son"] = df_q["Numéro du son"].astype(str)

<ipython-input-457-fcec55f414d2>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_q["Numéro du son"] = df_q["Numéro du son"].astype(str)


In [458]:
joueurs = ["MCK","PEPITO","BLEATZ","GOGO","SENTHS","WASSB","GOLSTO"] # se maj avec twitch
nb_j = len(joueurs)

#### Tableau score
Le tableau score, affiche le score de chaque participant à chaque question. Ajout une ligne automatiquement lorsqu'un nouveau joueur rejoint la partie, et donc commence avec 0 pts.

In [459]:
columns_score = ["Joueur", "Question"]
for i in list(bdd.columns[1:6]):  columns_score.append(i)
df_score = pd.DataFrame(columns=columns_score)    
df_score

,Joueur,Question,Nom de l'oeuvre,Titre du son,"Type du son (EP,ED,OST)",Numéro du son,Auteur du son


In [460]:
liste_q = []

for q in range(1,nb_q+1):
    for j in range(nb_j): 
        liste_q.append(q)   

In [461]:
df_score["Joueur"] = joueurs*nb_q
df_score["Question"] = liste_q

In [462]:
df_score[df_score["Question"] == 1]

,Joueur,Question,Nom de l'oeuvre,Titre du son,"Type du son (EP,ED,OST)",Numéro du son,Auteur du son
0,MCK,1,NaN,NaN,NaN,NaN,NaN
1,PEPITO,1,NaN,NaN,NaN,NaN,NaN
2,BLEATZ,1,NaN,NaN,NaN,NaN,NaN
3,GOGO,1,NaN,NaN,NaN,NaN,NaN
4,SENTHS,1,NaN,NaN,NaN,NaN,NaN
5,WASSB,1,NaN,NaN,NaN,NaN,NaN
6,GOLSTO,1,NaN,NaN,NaN,NaN,NaN


In [463]:
def get_chat_dataframe(file):
    data = []

    with open(file, 'r', encoding='utf-8') as f:
        lines = f.read().split('\n\n')
        for line in lines:
            try:
                time_logged = line.split('—')[0].strip()
                time_logged = datetime.strptime(time_logged, '%Y-%m-%d_%H:%M:%S')

                username_message = line.split('—')[1:]
                username_message = '—'.join(username_message).strip()

                username, channel, message = re.search(
                    ':(.*)\!.*@.*\.tmi\.twitch\.tv PRIVMSG #(.*) :(.*)', username_message
                ).groups()

                d = {
                    'dt': time_logged,
                    'channel': channel,
                    'username': username,
                    'message': message
                }

                data.append(d)
            
            except Exception:
                pass
            
    return pd.DataFrame().from_records(data)

In [464]:
df = get_chat_dataframe('chat_test.log')
df.set_index('dt', inplace=True)
print(df.shape)
df

(7, 3)


,channel,username,message
dt,,,
2022-03-19 17:31:01,ASENTHS,MCK,shingeki no kyojin s2/shinzou wo sasageyo!/op/...
2022-03-19 17:31:03,ASENTHS,PEPITO,shineki no kyojin s2/shinzou wo sasageyo!/op/1...
2022-03-19 17:31:04,ASENTHS,SENTHS,shingeki no kyojin s2/shinzou wo !/op/1/linked...
2022-03-19 17:31:06,ASENTHS,BLEATZ,shingeki no kyojin s2/shinzou wo sasageyo!/op/...
2022-03-19 17:31:09,ASENTHS,GOGO,shingeki no kyojin s2/shinzou wo sasageyo!/op/...
2022-03-19 17:31:10,ASENTHS,WASSB,shingeki no kyojin s2/shinzou wo sasageyo!/op/1
2022-03-19 17:31:12,ASENTHS,GOLSTO,shingeki no kyojin s2/shinzou wo sasageyo!/1/l...


#### Tableau réponse 

In [465]:
df_r = pd.DataFrame(columns=['username','answer'])

In [466]:
df_r['username'] = df['username']
df_r['answer'] = df['message']
df_r.reset_index(drop=True, inplace=True)

In [467]:
for it, elem in enumerate(df_r['answer']):
    elem_liste = elem.lower().split("/")
    elem_liste_clean = elem_liste
    for index, elem in enumerate(elem_liste):
        if isinstance(elem, str) == True:
            elem_liste_clean[index] = remove_all_extra_spaces(elem)
        else:
            elem_liste_clean[index] = elem
    df_r.loc[it,'answer'] = elem_liste_clean
df_r

,username,answer
0,MCK,"[shingeki no kyojin s2, shinzou wo sasageyo!, ..."
1,PEPITO,"[shineki no kyojin s2, shinzou wo sasageyo!, o..."
2,SENTHS,"[shingeki no kyojin s2, shinzou wo !, op, 1, l..."
3,BLEATZ,"[shingeki no kyojin s2, shinzou wo sasageyo!, ..."
4,GOGO,"[shingeki no kyojin s2, shinzou wo sasageyo!, ..."
5,WASSB,"[shingeki no kyojin s2, shinzou wo sasageyo!, ..."
6,GOLSTO,"[shingeki no kyojin s2, shinzou wo sasageyo!, ..."


In [475]:
df_r['username'][0]

'MCK'

#### Calculs des points

In [479]:
for i in range(nb_j):
    tot = 0
    for elem in df_r.loc[i,'answer']:
        distance, ratio = levCalclulate(elem, df_q["Titre du son"][1]) # nom du son
        if (distance > -2) and (distance <= 3) and (ratio > 0.8):
            print("Nom de l'oeuvre :", elem, " (+3 pts)")
            tot += 3

        distance, ratio = levCalclulate(elem, df_q["Nom de l\'oeuvre"][1]) # nom de l'oeuvre
        if (distance > -2) and (distance <= 3) and (ratio > 0.8):
            print("Titre du son :", elem, " (+1 pts)")
            tot += 1

        distance, ratio = levCalclulate(elem, df_q["Type du son (EP,ED,OST)"][1]) # OP/ED/OST
        if (distance > -2) and (distance <= 3) and (ratio > 0.8):
            print("Type du son (EP,ED,OST) :", elem, " (+1 pts)")
            tot += 0.5

        distance, ratio = levCalclulate(elem, df_q["Numéro du son"][1]) # Numéro
        if (distance > -2) and (distance <= 3) and (ratio > 0.8):
            print("Numéro du son :", elem, " (+1 pts)")
            tot += 0.5

        distance, ratio = levCalclulate(elem, df_q["Auteur du son"][1]) # Auteur
        if (distance > -2) and (distance <= 3) and (ratio > 0.8):
            print("Auteur du son :", elem, " (+1 pts)")
            tot += 1
    print("Total du joueur",df_r['username'][i]," : ",tot,"/ 7 pts \n")

Titre du son : shingeki no kyojin s2  (+1 pts)
Nom de l'oeuvre : shinzou wo sasageyo!  (+3 pts)
Type du son (EP,ED,OST) : op  (+1 pts)
Numéro du son : 1  (+1 pts)
Auteur du son : linked horizon  (+1 pts)
Total du joueur MCK  :  6.0 / 7 pts 

Titre du son : shineki no kyojin s2  (+1 pts)
Nom de l'oeuvre : shinzou wo sasageyo!  (+3 pts)
Type du son (EP,ED,OST) : op  (+1 pts)
Numéro du son : 1  (+1 pts)
Auteur du son : linked horizon  (+1 pts)
Total du joueur PEPITO  :  6.0 / 7 pts 

Titre du son : shingeki no kyojin s2  (+1 pts)
Type du son (EP,ED,OST) : op  (+1 pts)
Numéro du son : 1  (+1 pts)
Auteur du son : linked horizon  (+1 pts)
Total du joueur SENTHS  :  3.0 / 7 pts 

Titre du son : shingeki no kyojin s2  (+1 pts)
Nom de l'oeuvre : shinzou wo sasageyo!  (+3 pts)
Type du son (EP,ED,OST) : op  (+1 pts)
Numéro du son : 1  (+1 pts)
Auteur du son : linked horizon  (+1 pts)
Total du joueur BLEATZ  :  6.0 / 7 pts 

Titre du son : shingeki no kyojin s2  (+1 pts)
Nom de l'oeuvre : shinzou 

In [478]:
# bdd_01_score = pd.read_excel(r'BDD\bdd_score.xlsx')  

In [477]:
# bdd_01_score